<a href="https://colab.research.google.com/github/leech2193/weather_hospital_sowhat/blob/main/20220714_18_eseul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 2-2 날씨 빅데이터 콘테스트 unzip

In [ ]:
path_to_zip_file = '/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data/2-2날씨빅데이터 콘테스트.zip'
directory_to_extract_to = '/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data/2-2날씨빅데이터 콘테스트'

import zipfile

with zipfile.ZipFile(path_to_zip_file, 'r') as zf:
	zipinfo = zf.infolist()
	for info in zipinfo:
		info.filename = info.filename.encode('cp437').decode('euc-kr')
		zf.extract(info, directory_to_extract_to)

### 예보데이터 전처리 작업 코드 작성 (날짜 및 지역 반영)

In [1]:
import pandas as pd
import numpy as np

In [ ]:
path = '/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data/2-2날씨빅데이터 콘테스트/2012/강원/갈말읍_일최저기온_201201_201212.csv'
df = pd.read_csv(path, header=None)
df.columns = ['day','hour','forecast', 'value']
df.columns

Index(['day', 'hour', 'forecast', 'value'], dtype='object')

In [ ]:
df.loc[0, 'yyyymmdd'] = df.loc[0,'value'].strip()[-8:]
df.loc[df['day'].str.strip().str.startswith('Start'), 'yyyymmdd'] = df.loc[df['day'].str.strip().str.startswith('Start'), 'day'].str.strip().str[-8:]
df.yyyymmdd = df.yyyymmdd.fillna(method='ffill')
df.yyyymmdd = pd.to_datetime(df.yyyymmdd)
df['yyyy']     = df.yyyymmdd.dt.year
df['mm']       = df.yyyymmdd.dt.month
df['area'] = path.split('/')[-2]
df.rename(columns = {'value' : path.split('/')[-1].split('_')[-3]}, inplace=True)

drop_idx = df[(df['hour'].isna()) & (df['forecast'].isna())].index
df = df.drop(drop_idx).drop(0)

In [ ]:
df.head()

,day,hour,forecast,일최저기온,yyyymmdd,yyyy,mm,area
1,1,0200,+4,-13.900000,2012-01-01,2012,1,강원
2,1,0200,+7,-9.900000,2012-01-01,2012,1,강원
3,1,0500,+4,-13.900000,2012-01-01,2012,1,강원
4,1,0500,+7,-9.900000,2012-01-01,2012,1,강원
5,1,0800,+4,-12.900000,2012-01-01,2012,1,강원


In [ ]:
df.loc[494:505,:]

,day,hour,forecast,일최저기온,yyyymmdd,yyyy,mm,area
494,31,2000,+7,-19.900000,2012-01-01,2012,1,강원
495,31,2300,+4,-21.900000,2012-01-01,2012,1,강원
496,31,2300,+7,-19.900000,2012-01-01,2012,1,강원
498,1,0200,+4,-21.900000,2012-02-01,2012,2,강원
499,1,0200,+7,-19.900000,2012-02-01,2012,2,강원
500,1,0500,+4,-21.900000,2012-02-01,2012,2,강원
501,1,0500,+7,-19.900000,2012-02-01,2012,2,강원
502,1,0800,+4,-22.900000,2012-02-01,2012,2,강원
503,1,0800,+7,-19.900000,2012-02-01,2012,2,강원
504,1,1100,+4,-22.900000,2012-02-01,2012,2,강원


In [ ]:
import os
os.listdir('/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data/2-2날씨빅데이터 콘테스트')

['2012', '2013', '2014', '2015', '2016']

In [ ]:
import glob
glob.glob('/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data/2-2날씨빅데이터 콘테스트/*.csv', recursive=True)

[]

In [ ]:
file_lst = glob.glob('/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data/2-2날씨빅데이터 콘테스트/*/*/*.csv')
len(file_lst)
file_lst

In [ ]:
file_lst[0].split('/')[-1].split('_')[-3]

'12시간강수량'

### male, female concat

In [5]:
female = pd.read_csv('/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data/hospital_female.csv', encoding='cp949', index_col=0)
male = pd.read_csv('/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data/hospital_male.csv', encoding='cp949', index_col=0)
female.shape, male.shape

((24837, 24), (24837, 24))

In [ ]:
display(female.head())
display(male.head())

In [25]:
gender_lform = pd.concat([female, male])
gender_lform = gender_lform.sort_values(by=['area', 'tma'])
gender_lform.yyyy = gender_lform.yyyy.astype(int)
gender_lform.mm   = gender_lform.mm.astype(int)
gender_lform.dd   = gender_lform.dd.astype(int)
tmp = gender_lform[['sex','frequency']]
gender_lform.drop(['sex','frequency'], axis = 1, inplace=True)
gender_lform = pd.concat([gender_lform, tmp], axis = 1)
gender_lform = gender_lform.reset_index(drop=True)
gender_lform.head()

,area,tma,yyyy,mm,dd,avg_ta,max_ta,min_ta,sum_gsr,sum_rn,...,min_ps,so2,o3,pm10,elderly_pop,total_pop,elderly_ratio,avg_age,sex,frequency
0,강원,2012-01-01,2012,1,1,-4.226027,0.140278,-9.140278,0.0,0.173611,...,1020.193333,0.011685,0.026750,91.145833,137678.0,762617.0,18.053361,42.4,2,3
1,강원,2012-01-01,2012,1,1,-4.226027,0.140278,-9.140278,0.0,0.173611,...,1020.193333,0.011685,0.026750,91.145833,96281.0,772718.0,12.460044,39.4,1,3
2,강원,2012-01-02,2012,1,2,-6.504110,-0.559722,-11.276389,0.0,0.050000,...,1023.960000,0.008881,0.021179,82.200397,137678.0,762617.0,18.053361,42.4,2,0
3,강원,2012-01-02,2012,1,2,-6.504110,-0.559722,-11.276389,0.0,0.050000,...,1023.960000,0.008881,0.021179,82.200397,96281.0,772718.0,12.460044,39.4,1,2
4,강원,2012-01-03,2012,1,3,-6.215068,-0.566667,-11.344444,0.0,0.027778,...,1017.746667,0.009143,0.023786,96.958333,137678.0,762617.0,18.053361,42.4,2,0


In [26]:
gender_lform.to_csv('/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data/hospital_gender_longform.csv', encoding='cp949', index=False)